In [1]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import simpleaudio as sa

from plotly.subplots import make_subplots

empty    = np.empty(0, dtype=int)
wave_obj = sa.WaveObject.from_wave_file("sound.wav") # sound that is played at end of calculations

In [3]:
startfile  = 20
lastfile   = startfile
fileprefix = 'D'
filesuffix = ''
savefileStr = ''

showplot = False
saveplot = True
autoopen = True                                                  # If Save Plot is False Auto Open will not work

columnnames = ['Infected']                                       # This column is shown as a graph in the final file
allcolumns  = ['Infected','Healthy', 'Cured', 'Dead']

ShowVariance = False
AvgVarShow = True  # or False or 'legendonly'
AddVarShow = True  # or False or 'legendonly'

quantiles = [0.05, 0.25, 0.5, 0.75, 0.95]                      # Choose 5 Quantiles
qnames    = ['Min 5%', 'Q1', 'Median', 'Q2', 'Max 5%']         # Set Quantile Names

#quantiles = [0.025, 0.25, 0.5, 0.75, 0.975]                      # Choose 5 Quantiles
#qnames    = ['Min 2.5%', 'Q1', 'Median', 'Q2', 'Max 2.5%']       # Set Quantile Names
dashes    = ['dash', 'solid', 'dot', 'solid', 'dash']            # Set line styles
clsbright = ['rgba(0, 0, 255, 0.3)', 'rgba(255, 0, 0, 0.3)']     # Set Fillcolor between inner lines, Run 1 & Run 2
clsdim    = ['rgba(0, 0, 255, 0.1)', 'rgba(255, 0, 0, 0.1)']     # Set Fillcolor between outer lines, Run 1 & Run 2
clsbasic  = ['blue', 'red']                                      # Set Linecolor for Run 1 and Run 2.

rowMax = len(columnnames)
allMax = len(allcolumns)

for file in range(startfile,lastfile+1):
    data  = pd.read_csv('./Datasets/'+fileprefix+str(file)+filesuffix+'.csv')
    sdata = data.copy()
    
    title       = '<b>Quartile Lines</b> with outliers as scatter points - Dataset '+str(file)
    time        = data.columns.values.tolist()[0]
    duration    = int(data[time].max())

    ScatterData()
    Graph()

play_obj = wave_obj.play()
play_obj.wait_done()

In [2]:
def ScatterData():
    #sdata = pd.read_csv('./ModelData/'+fileprefix+str(file)+'.csv')
    for nd in (columnnames):
        for i in ([1,2]):
            lowerend = data.groupby(time).quantile(quantiles[0])[nd+str(i)].to_list()
            higherend = data.groupby(time).quantile(quantiles[4])[nd+str(i)].to_list()

            for j in range(1,duration):            
                mask = (sdata[time] == j) & (sdata[nd+str(i)] >= lowerend[j]) & (sdata[nd+str(i)] <= higherend[j])
                sdata.loc[mask,nd+str(i)] = None    
    return sdata

def AvgNames():
    str = ''
    for row in range(0,allMax):
        if allcolumns[row] != 'Dead': 
            str = str+allcolumns[row][:3]+'., '
    return str[:-2]

def rowspecs():
    spec = [[{"secondary_y": True}]]
    str = spec
    for row in range(0,rowMax):
        if row != 0: str = str+spec
    return str

def Graph():        
    total = np.zeros(duration)
    count = 0
    
    fig = make_subplots(rows=rowMax, cols=1, vertical_spacing = 0.15/rowMax, specs = rowspecs())
    fig.update_layout(height=rowMax*350+150, title_text=title)#, legend=dict( itemclick="toggleothers", itemdoubleclick="toggle"))
    
    for nd, row in zip(columnnames, range(1,rowMax+1)):
        if row == 1: showitem = True 
        else: showitem = False
        fig.update_yaxes(secondary_y = False, title_text = '<b>'+nd+' people</b>', gridwidth=2, rangemode='nonnegative', row=row, col=1)
        fig.update_yaxes(secondary_y = True,  title_text = "Percentage (%)", gridcolor='ghostwhite', row=row, col=1, range=[0, 100])
        if ShowVariance == False: 
            fig.update_yaxes(secondary_y = True, tickfont=dict(color='white'), title_font = {"color": 'white'})            
        fig.update_xaxes(range=[0, duration], row=row, title=time, col=1)
        fig['layout']['yaxis'+str(row*2)]['showgrid'] = False                
        
        for i, cls1 in zip([1, 2], clsbasic):
            
            fillColors  = ['black', clsdim[i-1], clsbright[i-1], clsbright[i-1], clsdim[i-1]]
            fillTo      = ['none','tonexty','tonexty', 'tonexty','tonexty']                
            
            for q, qn, fill, cls2, dashV in zip(quantiles, qnames, fillTo, fillColors, dashes):
                                
                fig.add_scatter( # --> Quartile Lines
                    y     = data.groupby(time).quantile(q)[nd+str(i)], 
                    mode  = 'lines', #marker= dict(symbol='diamond-tall'),# size=markersizeV), 
                    line  = dict(color=cls1, dash = dashV, width = 2), fillcolor = cls2, opacity=0.4, name = '<b>'+qn+'</b> (Run '+str(i)+')',legendgroup = str(i), showlegend=showitem, fill=fill, row=row, col=1)
                                    
            fig.add_scatter(     # --> Outleirs / Scatter
                y=sdata[nd+str(i)], x=sdata[time], mode = 'markers', name='<b>Outliers</b> (Run '+str(i)+')',
                marker_size=.2, marker_color=cls1, legendgroup = str(i), showlegend=showitem, row=row, col=1)#, hoverinfo='skip')
    
    """
    for nd, row in zip(allcolumns, range(1,allMax+1)):    
        var = np.round(np.multiply(np.divide(np.subtract(data.groupby(time).var()[nd+'1'],data.groupby(time).var()[nd+'2']),data.groupby(time).var()[nd+'1']),100)) #Calculate Added Variance
        if nd != 'Dead': 
            total = np.add(total,var)
            count = count + 1
        
        if nd in columnnames:
            fig.add_scatter(          # --> Added Variance
                y=var, mode='lines', row=row, col=1, secondary_y = True, name = 'Added Variance', legendgroup='Variance', showlegend=showitem, visible=AddVarShow, line=dict(color='orange', dash = 'dashdot'))
    
    totalavg = np.divide(total,count)
    
    for nd, row in zip(columnnames, range(1,rowMax+1)):
        if row == 1: showitem = True 
        else: showitem = False
        
        fig.add_scatter(      # --> Average Added Variance
            y=totalavg, mode="lines", 
            name='<b>Avg. Add. Var.</b>',# <i>('+AvgNames()+')</i>', 
            line=dict(width=2, color = 'darkslategrey', dash = 'dot'), 
            secondary_y=True, legendgroup='Avg', showlegend=showitem, visible=AvgVarShow, row=row, col=1)
    """
    
    fig.update_traces(selector=dict(mode='markers'), hoverinfo='none')    
    fig.update_traces(selector=dict(mode='lines'), hovertemplate = '<i> %{y:.1f} </i>')
    fig.update_traces(selector=dict(line=dict(color='grey', dash = 'dashdot')), hovertemplate = '<i> %{y}% </i>')
    fig.update_layout(hovermode = 'x unified')
    
    savecolumns = ','.join(map(str, columnnames))
    if ShowVariance == False: saveVar = '- HiddenVar'
    else: ''
    
    if showplot == True: fig.show()
    if saveplot == True: plotly.offline.plot(fig, filename='./ModelGraphsNew/'+fileprefix+str(file)+'.LineGraphs+Scatter'+savefileStr+'-'+savecolumns+'.html', auto_open=autoopen)